# 07 - DecisionTree

Realizando a predição com DecisionTree. Lembrando que a acurácia para os dados de treino precisa ser superior a 78% para que seja considerada válida para testarmos com os dados de teste.

## Preparando o ambiente

In [1]:
#!pip install graphviz
#!conda install python-graphviz

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler
import graphviz
from IPython.display import display

print(f'GraphViz: {graphviz.__version__}')

GraphViz: 0.17


In [3]:
sns.set(rc={'figure.figsize':(18, 12)})

In [4]:
def exibir_arvore(modelo):
  dot_data = export_graphviz(modelo, 
                  out_file=None,
                  filled=True,
                  rounded=True)
  display(graphviz.Source(dot_data))

## Carregando os dados

### Carregando dados de treino

In [49]:
titanic = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed/train_processed_byage_ag.csv')
del titanic['Unnamed: 0']
titanic.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,male,22,1,0,SC,1,jovem_adulto,True,False
1,0,3,male,22,0,0,SC,0,jovem_adulto,False,False
2,0,3,male,22,0,0,SC,0,jovem_adulto,False,False
3,0,3,male,22,0,0,SC,0,jovem_adulto,False,False
4,0,3,male,22,0,0,SC,0,jovem_adulto,False,False


In [50]:
sex_map = {
    'male': 0,
    'female': 1
}
titanic['Sex'] = titanic['Sex'].map(sex_map)
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,SC,1,jovem_adulto,True,False
1,0,3,0,22,0,0,SC,0,jovem_adulto,False,False


In [51]:
boolean_map = {
    True: 1,
    False: 0
}
titanic['acompanhado'] = titanic['acompanhado'].map(boolean_map)
titanic['possui_cabine'] = titanic['possui_cabine'].map(boolean_map)
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,SC,1,jovem_adulto,1,0
1,0,3,0,22,0,0,SC,0,jovem_adulto,0,0


In [52]:
titanic['Floor'].unique()

array(['SC', 'C', 'E', 'B', 'A', 'D', 'G', 'F', 'T'], dtype=object)

In [53]:
floor_map = {
    'SC': 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
    'T': 8
}
titanic['Floor'] = titanic['Floor'].map(floor_map)
titanic.isnull().sum()

Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Floor            0
Relateds         0
faixa_etaria     0
acompanhado      0
possui_cabine    0
dtype: int64

In [54]:
faixa_etaria_map = {
    'jovem_adulto': 1,
    'adulto_idoso': 2,
    'crianca_adolescente': 0
}
titanic['faixa_etaria'] = titanic['faixa_etaria'].map(faixa_etaria_map)
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,0,1,1,1,0
1,0,3,0,22,0,0,0,0,1,0,0


In [55]:
treino_1 = titanic
treino_1.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
61,0,3,0,26,0,0,0,0,1,0,0
210,1,1,1,39,1,1,3,2,1,1,1


### Carregando dados de teste

In [56]:
teste = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed/test_processed.csv')
del teste['Unnamed: 0']

teste['acompanhado'] = teste['acompanhado'].map(boolean_map)
teste['possui_cabine'] = teste['possui_cabine'].map(boolean_map)
teste['Floor'] = teste['Floor'].map(floor_map)
teste['Sex'] = teste['Sex'].map(sex_map)
teste['faixa_etaria'] = teste['faixa_etaria'].map(faixa_etaria_map)

Xvalidacao = teste.drop(columns=['PassengerId'])
Xvalidacao.head(5)

,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,possui_cabine,acompanhado,faixa_etaria
0,3,0,34,0,0,0,0,0,0,1
1,3,1,47,1,0,0,1,0,1,2
2,2,0,62,0,0,0,0,0,0,2
3,3,0,27,0,0,0,0,0,0,1
4,3,1,22,1,1,0,2,0,1,1


### Separando treino e teste

In [57]:
SEED = 5
np.random.seed(SEED)

In [58]:
X = treino_1.drop(columns=['Survived'])
y = treino_1['Survived']

In [59]:
treino_x, teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.2, stratify = y)

In [60]:
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 712 elementos e testaremos com 179 elementos


## Execução da Árvore de decisão

In [87]:
espaco_de_parametros = {
    "criterion": ["gini", "entropy"],
    "max_depth" : [2, 3, 5, 7, 9, 11, 13],
    "min_samples_split": [2, 8, 16, 32],
    "min_samples_leaf": [1, 2, 8, 16, 32]    
}

In [88]:
busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)

In [89]:
busca.fit(treino_x, treino_y)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 7, 9, 11, 13],
                         'min_samples_leaf': [1, 2, 8, 16, 32],
                         'min_samples_split': [2, 8, 16, 32]})

In [90]:
resultados = pd.DataFrame(busca.cv_results_)
resultados.sample(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
157,0.003126,0.006252,0.00313,0.006260,entropy,2,32,8,"{'criterion': 'entropy', 'max_depth': 2, 'min_...",0.755245,0.762238,0.767606,0.753521,0.830986,0.773919,0.028975,240
26,0.001602,0.003204,0.00601,0.003124,gini,3,2,16,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.811189,0.818182,0.753521,0.809859,0.830986,0.804747,0.026686,30


In [91]:
print(busca.best_params_)
print(busca.best_score_ * 100)

{'criterion': 'gini', 'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8}
82.16980202895697


In [92]:
busca.best_estimator_.score(teste_x, teste_y) * 100

84.91620111731844

A acurácia obtida de 86.2% já foi melhor do que o classificador manual, mas ainda não satisfatoriamente melhor.

## Normalizando os dados

Realizando os testes normalizando através de: `StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler`

### `StandardScaler`

In [93]:
scaler = StandardScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

In [94]:
busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)

{'criterion': 'gini', 'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8}
81.46656160740669


In [95]:
busca.best_estimator_.score(n_teste_x, teste_y) * 100

84.91620111731844

`MaxAbsScaler`

In [96]:
scaler = MaxAbsScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 2, 'min_samples_split': 8}
81.8871269575495
84.35754189944134


In [97]:
scaler = MinMaxScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8}
81.74825174825176
84.91620111731844


In [98]:
scaler = Normalizer()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 8}
79.07712006303555
81.00558659217877


In [100]:
scaler = RobustScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8}
82.16980202895697
84.91620111731844
0.8491620111731844


Mesmo com a normalização, a acurácia se manteve em **86.0%**